# Importing the Libraries

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
import matplotlib.pyplot as plt
%matplotlib inline
import random

C:\Users\Brian\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Loading and looking at the Data

In [3]:
data = pd.read_csv('Iris.csv')

In [4]:
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
data.describe()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,75.500000,5.843333,3.054000,3.758667,1.198667
std,43.445368,0.828066,0.433594,1.764420,0.763161
min,1.000000,4.300000,2.000000,1.000000,0.100000
25%,38.250000,5.100000,2.800000,1.600000,0.300000
50%,75.500000,5.800000,3.000000,4.350000,1.300000
75%,112.750000,6.400000,3.300000,5.100000,1.800000
max,150.000000,7.900000,4.400000,6.900000,2.500000


In [4]:
data.shape

(150, 6)

In [14]:
x_data = data.drop(['Species'], axis=1)
y_data = data['Species']

In [17]:
type(y_data)

pandas.core.series.Series

In [6]:
y_data.unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

# OneHotEncoding the labels and scaling the data

In [18]:
encoder = LabelEncoder()

In [19]:
label_encoded = encoder.fit(y_data).transform(y_data)

In [25]:
label_encoded.shape

(150,)

In [22]:
label_encoded

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [12]:
hotencoder = OneHotEncoder(3)

In [13]:
y_data = hotencoder.fit_transform(label_encoded).toarray().reshape(150,3)

C:\Users\Brian\Anaconda3\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [191]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1)

In [192]:
scalar = MinMaxScaler()

In [193]:
scalar.fit(X_train)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [194]:
X_train = pd.DataFrame(scalar.transform(X_train), index=X_train.index, columns=X_train.columns)
X_test = pd.DataFrame(scalar.transform(X_test), index=X_test.index, columns=X_test.columns)

# Creating the TensorFlow Model

In [195]:
y_true = tf.placeholder(tf.float32)
x = tf.placeholder(tf.float32)

In [196]:
#Weights are 5*3
W = tf.Variable(tf.random_uniform([5,3]))
b = tf.Variable(tf.random_uniform([3]))

In [197]:
y_pred = tf.add(tf.matmul(x,W), b)

In [198]:
error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [199]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=.1).minimize(error)

In [200]:
init = tf.global_variables_initializer()

In [228]:
num_epochs = 1000

# Creating a custom next_batch function

In [204]:
def next_batch(x, y, batch_size=50):
    '''
    returns a batch from x and y respectivally of size batch_size
    '''
    start = int(random.random()*x.shape[0])
    if start<x.shape[0]-1-batch_size:
        x_batch = x[start:start+batch_size]
        y_batch = y[start:start+batch_size]
    else:
        rollover = batch_size-(x.shape[0]-start-1)
        x_batch = pd.concat([x[start:], x[:rollover]])
        y_batch = np.concatenate((y[start:], y[:rollover]))
        
    return x_batch, y_batch

# Running the TensorFlow session

In [229]:
with tf.Session() as sess:
    
    sess.run(init)
    accs = []
    epoch_list=[]
    
    for epoch in range(num_epochs):
        
        x_batch, y_batch = next_batch(X_train, y_train)
        
        sess.run(optimizer, feed_dict = {x:x_batch, y_true:y_batch})
        
        if epoch%10==0:
            print('Epoch number {}'.format(epoch))

            
            preds = sess.run(y_pred, feed_dict={x: X_train})
                        
            acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(preds, 1), tf.argmax(y_train, 1)), tf.float32))
            acc = sess.run(acc)
            
            print(acc)

            accs.append(acc)
            epoch_list.append(epoch)
            plt.plot(epoch_list, accs)
            plt.show()
            

    
    preds = sess.run(y_pred, feed_dict={x: X_train})
    acc_train = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(preds, 1), tf.argmax(y_train, 1)), tf.float32))
    acc_train = sess.run(acc_train)
    
    preds = sess.run(y_pred, feed_dict={x: X_test})
    acc_test = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(preds, 1), tf.argmax(y_test, 1)), tf.float32))
    acc_test = sess.run(acc_test)
    
    print("Final rain Accuracy = {}".format(acc_train))
    print("Test accuracy = {}".format(acc_test))

    final_w, final_b = sess.run([W, b])

Epoch number 0
Epoch number 10
Epoch number 20
Epoch number 30
Epoch number 40
Epoch number 50
Epoch number 60
Epoch number 70
Epoch number 80
Epoch number 90
Epoch number 100
Epoch number 110
Epoch number 120
Epoch number 130
Epoch number 140
Epoch number 150
Epoch number 160
Epoch number 170
Epoch number 180
Epoch number 190
Epoch number 200
Epoch number 210
Epoch number 220
Epoch number 230
Epoch number 240
Epoch number 250
Epoch number 260
Epoch number 270
Epoch number 280
Epoch number 290
Epoch number 300
Epoch number 310
Epoch number 320
Epoch number 330
Epoch number 340
Epoch number 350
Epoch number 360
Epoch number 370
Epoch number 380
Epoch number 390
Epoch number 400
Epoch number 410
Epoch number 420
Epoch number 430
Epoch number 440
Epoch number 450
Epoch number 460
Epoch number 470
Epoch number 480
Epoch number 490
Epoch number 500
Epoch number 510
Epoch number 520
Epoch number 530
Epoch number 540
Epoch number 550
Epoch number 560
Epoch number 570
Epoch number 580
Epoch nu